# Mixture-Aware Encoding Model

**Problem**: CV-LB intercept (0.0525) > Target (0.0347). Linear interpolation of mixture features misses non-linear effects.

**Solution**: Learn mixture effects with a neural network instead of linear interpolation.

**Key insight from benchmark**: The benchmark achieved MSE 0.0039 using learned mixture-aware encodings.

**This notebook has EXACTLY 3 submission cells at the end.**

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.float32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

# Create combined solvent features
SOLVENT_FEATURES = pd.concat([SPANGE_DF, DRFP_FILTERED, ACS_PCA_DF], axis=1)
print(f'Combined solvent features: {SOLVENT_FEATURES.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)
Combined solvent features: (26, 140)


In [4]:
# Mixture-Aware Encoder
class MixtureAwareEncoder(nn.Module):
    """Learn mixture effects instead of linear interpolation."""
    
    def __init__(self, solvent_dim, hidden_dim=64):
        super().__init__()
        # Encode each solvent separately
        self.solvent_encoder = nn.Sequential(
            nn.Linear(solvent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        # Learn mixture interaction (non-linear!)
        self.mixture_net = nn.Sequential(
            nn.Linear(hidden_dim * 2 + 1, hidden_dim),  # +1 for percentage
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
    
    def forward(self, solvent_a_features, solvent_b_features, pct_b):
        enc_a = self.solvent_encoder(solvent_a_features)
        enc_b = self.solvent_encoder(solvent_b_features)
        # Concatenate encodings with percentage
        mixture_input = torch.cat([enc_a, enc_b, pct_b.unsqueeze(-1)], dim=-1)
        return self.mixture_net(mixture_input)

print('MixtureAwareEncoder defined')

MixtureAwareEncoder defined


In [5]:
# Mixture-Aware Model for yield prediction
class MixtureAwareModel(nn.Module):
    """Model with learned mixture-aware encoding."""
    
    def __init__(self, solvent_dim, kinetics_dim=5, hidden_dim=64):
        super().__init__()
        self.mixture_encoder = MixtureAwareEncoder(solvent_dim, hidden_dim)
        
        # For single solvent, just encode the solvent
        self.single_encoder = nn.Sequential(
            nn.Linear(solvent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Prediction head
        self.predictor = nn.Sequential(
            nn.Linear(hidden_dim + kinetics_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, 3),
            nn.Sigmoid()
        )
    
    def forward(self, solvent_features, kinetics, pct=None, solvent_b_features=None):
        if solvent_b_features is not None and pct is not None:
            # Mixed solvent case
            solvent_encoding = self.mixture_encoder(solvent_features, solvent_b_features, pct)
        else:
            # Single solvent case
            solvent_encoding = self.single_encoder(solvent_features)
        
        # Combine with kinetics
        combined = torch.cat([solvent_encoding, kinetics], dim=-1)
        return self.predictor(combined)

print('MixtureAwareModel defined')

MixtureAwareModel defined


In [6]:
# Wrapper class for training and prediction
class MixtureAwareModelWrapper:
    """Wrapper for MixtureAwareModel with training and prediction."""
    
    def __init__(self, data='single', hidden_dim=64, n_models=3):
        self.data_type = data
        self.hidden_dim = hidden_dim
        self.n_models = n_models
        self.models = []
        self.scaler = StandardScaler()
        
    def _get_kinetics(self, X):
        """Extract kinetics features."""
        time_m = X["Residence Time"].values.reshape(-1, 1)
        temp_c = X["Temperature"].values.reshape(-1, 1)
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        return np.hstack([time_m, temp_c, inv_temp, log_time, interaction]).astype(np.float32)
    
    def _get_solvent_features(self, solvent_names):
        """Get solvent features from lookup."""
        return SOLVENT_FEATURES.loc[solvent_names].values.astype(np.float32)
    
    def train_model(self, X_train, Y_train, epochs=200):
        Y_np = Y_train.values if hasattr(Y_train, 'values') else Y_train
        
        # Get features
        kinetics = self._get_kinetics(X_train)
        kinetics_scaled = self.scaler.fit_transform(kinetics)
        
        if self.data_type == 'single':
            solvent_features = self._get_solvent_features(X_train["SOLVENT NAME"])
            solvent_b_features = None
            pct = None
        else:
            solvent_features = self._get_solvent_features(X_train["SOLVENT A NAME"])
            solvent_b_features = self._get_solvent_features(X_train["SOLVENT B NAME"])
            pct = X_train["SolventB%"].values.astype(np.float32)
        
        # Convert to tensors
        kinetics_tensor = torch.tensor(kinetics_scaled, dtype=torch.float32).to(device)
        solvent_tensor = torch.tensor(solvent_features, dtype=torch.float32).to(device)
        Y_tensor = torch.tensor(Y_np, dtype=torch.float32).to(device)
        
        if solvent_b_features is not None:
            solvent_b_tensor = torch.tensor(solvent_b_features, dtype=torch.float32).to(device)
            pct_tensor = torch.tensor(pct, dtype=torch.float32).to(device)
        else:
            solvent_b_tensor = None
            pct_tensor = None
        
        # Train multiple models
        self.models = []
        solvent_dim = solvent_features.shape[1]
        
        for seed in range(self.n_models):
            torch.manual_seed(42 + seed)
            
            model = MixtureAwareModel(solvent_dim, kinetics_dim=5, hidden_dim=self.hidden_dim).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            
            model.train()
            batch_size = 32
            n_samples = len(Y_np)
            
            for epoch in range(epochs):
                indices = torch.randperm(n_samples)
                for start in range(0, n_samples, batch_size):
                    end = min(start + batch_size, n_samples)
                    batch_idx = indices[start:end]
                    
                    batch_kinetics = kinetics_tensor[batch_idx]
                    batch_solvent = solvent_tensor[batch_idx]
                    batch_y = Y_tensor[batch_idx]
                    
                    if solvent_b_tensor is not None:
                        batch_solvent_b = solvent_b_tensor[batch_idx]
                        batch_pct = pct_tensor[batch_idx]
                        pred = model(batch_solvent, batch_kinetics, batch_pct, batch_solvent_b)
                    else:
                        pred = model(batch_solvent, batch_kinetics)
                    
                    optimizer.zero_grad()
                    loss = criterion(pred, batch_y)
                    loss.backward()
                    optimizer.step()
            
            model.eval()
            self.models.append(model)
    
    def predict(self, X_test):
        kinetics = self._get_kinetics(X_test)
        kinetics_scaled = self.scaler.transform(kinetics)
        kinetics_tensor = torch.tensor(kinetics_scaled, dtype=torch.float32).to(device)
        
        if self.data_type == 'single':
            solvent_features = self._get_solvent_features(X_test["SOLVENT NAME"])
            solvent_tensor = torch.tensor(solvent_features, dtype=torch.float32).to(device)
            solvent_b_tensor = None
            pct_tensor = None
        else:
            solvent_features = self._get_solvent_features(X_test["SOLVENT A NAME"])
            solvent_b_features = self._get_solvent_features(X_test["SOLVENT B NAME"])
            solvent_tensor = torch.tensor(solvent_features, dtype=torch.float32).to(device)
            solvent_b_tensor = torch.tensor(solvent_b_features, dtype=torch.float32).to(device)
            pct_tensor = torch.tensor(X_test["SolventB%"].values.astype(np.float32), dtype=torch.float32).to(device)
        
        preds = []
        with torch.no_grad():
            for model in self.models:
                if solvent_b_tensor is not None:
                    pred = model(solvent_tensor, kinetics_tensor, pct_tensor, solvent_b_tensor)
                else:
                    pred = model(solvent_tensor, kinetics_tensor)
                preds.append(pred.cpu())
        
        return torch.clamp(torch.stack(preds).mean(dim=0), 0, 1)

print('MixtureAwareModelWrapper defined')

MixtureAwareModelWrapper defined


In [7]:
# Quick test on single fold
X_single, Y_single = load_data("single_solvent")
test_solvent = sorted(X_single["SOLVENT NAME"].unique())[0]
mask = X_single["SOLVENT NAME"] != test_solvent

print(f"Test solvent: {test_solvent}")
print(f"Training samples: {mask.sum()}, Test samples: {(~mask).sum()}")

model = MixtureAwareModelWrapper(data='single', n_models=2)
model.train_model(X_single[mask], Y_single[mask], epochs=100)
preds = model.predict(X_single[~mask])

actuals = Y_single[~mask].values
mse = np.mean((actuals - preds.numpy()) ** 2)
print(f'Test fold MSE: {mse:.6f}')

Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol
Training samples: 619, Test samples: 37


Test fold MSE: 0.024847


In [8]:
# Run full CV for single solvent
print('Running CV for single solvent data...')

X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_preds = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = MixtureAwareModelWrapper(data='single', n_models=3)
    model.train_model(train_X, train_Y, epochs=150)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    all_preds.append(predictions_np)
    all_actuals.append(test_Y.values)

all_preds = np.vstack(all_preds)
all_actuals = np.vstack(all_actuals)
mse_single = np.mean((all_preds - all_actuals) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')

Running CV for single solvent data...


0it [00:00, ?it/s]

1it [00:10, 10.09s/it]

2it [00:20, 10.11s/it]

3it [00:29,  9.89s/it]

...

22it [03:39,  9.97s/it]

23it [03:49,  9.94s/it]

24it [03:59,  9.99s/it]

24it [03:59,  9.97s/it]


Single Solvent MSE: 0.008904


In [ ]:
# Run full CV for full data
print('Running CV for full data...')

X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_preds_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = MixtureAwareModelWrapper(data='full', n_models=3)
    model.train_model(train_X, train_Y, epochs=150)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    all_preds_full.append(predictions_np)
    all_actuals_full.append(test_Y.values)

all_preds_full = np.vstack(all_preds_full)
all_actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((all_preds_full - all_actuals_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')

# Overall MSE
n_single = len(all_actuals)
n_full = len(all_actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)
print(f'\nOverall MSE: {overall_mse:.6f}')
print(f'Baseline (exp_030 GP+MLP+LGBM): CV 0.008298')

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MixtureAwareModelWrapper(data='single', n_models=3)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MixtureAwareModelWrapper(data='full', n_models=3)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################